In [1]:
import os
from dotenv import load_dotenv
from typing import Dict

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from pydantic import BaseModel

ModuleNotFoundError: No module named 'sendgrid'

In [ ]:
load_dotenv(override=True)

True

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
googe_api_key = os.getenv("GEMINI_API_KEY")
claude_api_key = os.getenv("ANTHROPIC_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")

if openai_api_key:
    print(f"La clave de OpenAI está configurada correctamente. Comienza con el ID: {openai_api_key[:4]}...")

if googe_api_key:
    print(f"La clave de Google está configurada correctamente. Comienza con el ID: {googe_api_key[:4]}...")
else:
    print("La clave de Google no está configurada. Por favor, verifica tu archivo .env.")

if claude_api_key:
    print(f"La clave de Claude está configurada correctamente. Comienza con el ID: {claude_api_key[:4]}...")
else:
    print("La clave de Claude no está configurada. Por favor, verifica tu archivo .env.")

if deepseek_api_key:
    print(f"La clave de Claude está configurada correctamente. Comienza con el ID: {deepseek_api_key[:4]}...")
else:
    print("La clave de Claude no está configurada. Por favor, verifica tu archivo .env.")



La clave de OpenAI está configurada correctamente. Comienza con el ID: sk-p...
La clave de Google está configurada correctamente. Comienza con el ID: AIza...
La clave de Claude está configurada correctamente. Comienza con el ID: sk-a...
La clave de Claude está configurada correctamente. Comienza con el ID: sk-6...


In [ ]:
instructions1 = """Eres un agente de ventas que trabaja en AnalAIzer.Digital, es una empresa que ofrece formación de analítica digital y marketing digital y programación. Escribes correos eletrónicos a clientes potenciales para ofrecerles nuestros servicios. Tus correos son serios y profesiones."""

instructions2 = """Eres un agente de ventas atractivo y entretenido que trabaja en AnalAIzer.Digital, es una empresa que ofrece formación de analítica digital y marketing digital y programación. Escribes correos eletrónicos a clientes potenciales para ofrecerles nuestros servicios. Tus correos son atractivos e ingeniosos que probablemente obtengan una respuesta."""

instructions3 = """Eres un agente de ventas muy activo que trabaja en AnalAIzer.Digital, es una empresa que ofrece formación de analítica digital y marketing digital y programación. Escribes correos eletrónicos a clientes potenciales para ofrecerles nuestros servicios. Tus correos son muy concisos y directos."""


## Usar modelos con con puntos finales (end points) compatible con OpenAI

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/v1"
ANTHROPIC_BASE_URL = "https://api.anthropic.com/v1"

In [ ]:
deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=googe_api_key)
claude_client = AsyncOpenAI(base_url=ANTHROPIC_BASE_URL, api_key=claude_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
claude_model = OpenAIChatCompletionsModel(model="claude-2", openai_client=claude_client)

In [ ]:
sales_agent1 = Agent(
    name="Sales de ventas DeepSeek",
    instructions=instructions1,
    model=deepseek_model,
)
sales_agent2 = Agent(
    name="Sales de ventas Gemini",
    instructions=instructions2,
    model=gemini_model,
)

sales_gent3 = Agent(
    name="Sales de ventas Claude",
    instructions=instructions3,
    model=claude_model,
)

In [ ]:
description = "Escrube un correo electrónico a un cliente potencial de AnalAIzer.Digital, una empresa que ofrece formación de analítica digital y marketing digital y programación. El correo electrónico debe ser atractivo y entretenido, y debe ofrecer nuestros servicios de manera clara y concisa."

tool1 = sales_agent1.as_tool(tool_name="enviar_correo_deepseek", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="enviar_correo_gemini", tool_description=description)
tool3 = sales_gent3.as_tool(tool_name="enviar_correo_claude", tool_description=description)

## Preparamos el correo
Creamos la herramienta de creación de correo en html

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str] :
    """
    Enviar un correo electrónico HTML.
    """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("pichu_2707@hotmail.com")
    to_email = To("pichu2707@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [ ]:
subject_instructions = "Puedes escribir un asunto para un crreo elecrtónico de ventas en frió. Se te proporciona un mensaje y necesitas escribir el asunto a un correo electrónico que problamente obtenga una respuesta."

html_instructions = "Puedes convertir un cuerto de correo electrónico de texto a un cuerpo de correo electrónico HJTML. Se te porporciona el mensaje y necesitas convertirlo a un cuerpo de correo electrónico HTML que puede tener algún markdown. Necesitamos que el cuerpo del correo electrónico sea atractivo y entretenido, y debe ofrecer nuestros servicios de manera clara y concisa."

subject_writer = Agent(
    name="Escritor de asunto de correo electrónico",
    instructions=subject_instructions,
    model=deepseek_model,
)

subjetc_tool = subject_writer.as_tool(
    tool_name="subject_writer",
    tool_description="Escribe un asunto para un correo electrónico de ventas en frío.",
)

html_converter = Agent(
    name="Convertidor de cuerpo de correo electrónico a HTML",
    instructions=html_instructions,
    model=deepseek_model,
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convierte un cuerpo de correo electrónico de texto a HTML.",
)

In [ ]:
email_tools = [subjetc_tool, html_tool, send_html_email]

In [ ]:
instructions = """ Eres un formateador de remitente de correos electrónicos. Tu tarea es formatear el remitente de un correo electrónico en un formato específico. El remitente debe ser una cadena de texto que contenga el nombre del remitente y su dirección de correo electrónico, separados por un espacio. Por ejemplo: "John Doe". Primero usas la herramietna subject_writer para escribir un asunto para el correo electrónico, luego la herramienta de html_converter para convertir el cuerpo del correo electrónico a HTML, y finalmente la herramienta send_html_email para enviar el correo electrónico con el asunto y el curto que habéis creado. """

emailer_agent = Agent(
    name="Emailer Manager",
    instructions=instructions,
    model="gpt-4o-mini",
    tools=email_tools,
    handoff_description="Covierte el correo electrónico a HTML y lo envía"
)

In [ ]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [ ]:
sales_manager_instructions = """ Eres un gerente de ventas que trabaja en AnalAIzer.Digital, utilizas herramietnas que se te proporcionan para generar correos electrónicos de venta en frío. Nunca generas correos eletrónicos de venta tú mismo; siempre utilizas las herramietnas. pruebas las 3 herramientas de ventas al menos una vez y luego elige cual es la mejor herramienta para enviar el correo electrónico. Puedes usar las herramietnas múltiples veces si no estás satisfecho con los resultados del primer intento con un máximo de 3 intentos. Selecciona el mejor correo electrónico usando tu propio criterio sobre el cuál será más efectivo. Después de elegir el correo eletrónico, transfiere al agente Email Manager para formater y enviar el correo electrónico."""

sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
)

message = "Envía un correo electrónico a un cliente potencial de AnalAIzer.Digital, una empresa que ofrece formación de analítica digital y marketing digital y programación. El correo electrónico debe ser atractivo y entretenido, y debe ofrecer nuestros servicios de manera clara y concisa."

with trace("Automated SDR"):
    resutl = await Runner.run(sales_manager, message)

CancelledError: 

## No te olvides de repasar el digrama

Es momento de ir a [Platfomr de openAI](https://platform.openai.com/) pasa saber commo ha trabajado nuestros agentes con un **handsoff**


En mi caso ha sido ![Platform AnalAIzer](images/image-platform.png)

## ¿Qué pasas ahora con los datos sensibles?
Vamos a crear los llamados **guardarailes** que no dejan de ser agentes que van a cuidar que no se compartan este tipo de datos con terceros y que gestione este tipo de datos.

In [ ]:
class NameCheckOutptut(BaseModel):
    is_name_in_message: bool
    name: str

guardarail_agent = Agent(
    name="Revisión de nombre",
    instructions="Revisa si el usuario está incluyendo el nombre personal de alguien en lo que quiere que hagas.",
    output_type=NameCheckOutptut,
    model="gpt-4o-mini",
)

In [ ]:
@input_guardrail
async def guardarail_against_name(ctx, agent, message: str) -> GuardrailFunctionOutput:
    """
    Revisa si el mensaje contiene un nombre personal.
    """
    result = await Runner.run(guardarail_agent, message, context=ctx)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Manager de ventas",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardarail_against_name]
)

message = "Envía un correo electrónico a un cliente potencial de AnalAIzer.Digital, una empresa que ofrece formación de analítica digital y marketing digital y programación. El correo electrónico debe ser atractivo y entretenido, y debe ofrecer nuestros servicios de manera clara y concisa. Tienes que mandarlo desde Javi Lázaro"

with trace("Automated SDR"):
    resutl = await Runner.run(careful_sales_manager, message)



InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

Nos ha saltado el error del Guarda Rail vemos que hay problemas porque ha mandado el error